In [68]:
import importlib
import shapelets as sha
importlib.reload(sha)
import os
import fnmatch
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from collections import Counter

Load the observation ids of classifications from Huppenkothen et al. and create an ob_state dictionary of (presumed) file names, which contain data of those observations, against the classification.

In [55]:
ob_state=sha.import_labels('1915Belloniclass_updated.dat', "_std1_lc.txt")
clean_belloni = open('1915Belloniclass_updated.dat')
lines = clean_belloni.readlines()
states = lines[0].split()
belloni_clean = {}
for h,l in zip(states, lines[1:]):
    belloni_clean[h] = l.split()
    #state: obsID1, obsID2...
ob_state = {}
for state, obs in belloni_clean.items():
    if state == "chi1" or state == "chi2" or state == "chi3" or state == "chi4": state = "chi"
    for ob in obs:
        ob_state[ob] = state

Count the number of classifications for every class.
Only classes with an acceptable number of samples should be analysed. Let's make the acceptable number at least 7.

In [12]:
states=[]
for obID, state in ob_state.items():
    if state not in states:
        states.append(state)
counts=np.zeros(len(states))
for obID, state in ob_state.items():
    if obID in pool:
        counts[np.where(np.array(states)==state)]+=1
state_count=[]
for i, state in enumerate(states):
    state_count.append((state,counts[i]))
state_count

[('alpha', 8.0),
 ('beta', 16.0),
 ('gamma', 12.0),
 ('delta', 23.0),
 ('theta', 19.0),
 ('kappa', 12.0),
 ('lambda', 4.0),
 ('mu', 7.0),
 ('nu', 3.0),
 ('rho', 19.0),
 ('phi', 9.0),
 ('chi', 77.0),
 ('eta', 3.0),
 ('omega', 3.0)]

Check the availability of the files from ob_state and append their names to the pool.

In [56]:
available = []
pool=[]
for root, dirnames, filenames in os.walk("/home/jkok1g14/Documents/GRS1915+105/data/Std1_PCU2"):
    for filename in fnmatch.filter(filenames, "*_std1_lc.txt"):
        available.append(filename)
for ob, state in ob_state.items():
    if ob+"_std1_lc.txt" in available:
        pool.append(ob)  
len(pool)

215

Create arrays of light curve time series.

In [106]:
#create a list of arrays with time and counts for the set of Belloni classified observations
lc_dirs=[]
lcs=[]
ids=[]
for root, dirnames, filenames in os.walk("/home/jkok1g14/Documents/GRS1915+105/data/Std1_PCU2"):    
    for filename in fnmatch.filter(filenames, "*_std1_lc.txt"):
        if filename.split("_")[0] in pool:
            lc_dirs.append(os.path.join(root, filename))

            
#make 2D arrays for light curves, with columns of counts and time values
for lc in lc_dirs:
    ids.append(lc.split("/")[-1].split("_")[0])
    f=np.loadtxt(lc)
    f=np.transpose(f)#,axis=1)
    f=f[0:2]
    ###1s average and time check to eliminate points outside of GTIs
    f8t = np.mean(f[0][:(len(f[0])//8)*8].reshape(-1, 8), axis=1)
    f8c = np.mean(f[1][:(len(f[1])//8)*8].reshape(-1, 8), axis=1)
    f8c=f8c/np.max(f8c)
    rm_points = []
    skip=False
    for i in range(len(f8t)-1):
        if skip==True:
            skip=False
            continue
        delta = f8t[i+1]-f8t[i]
        if delta > 1.0:
            rm_points.append(i+1)
            skip=True
            
####### normalise the count rates! think about the effect of 0-1 normalisation on the distance calculation
            
    times=np.delete(f8t,rm_points)
    counts=np.delete(f8c,rm_points)
    lcs.append(np.stack((times,counts)))

Create an ordered list of classifications from the order ids list and the ob_state dict.

In [115]:
lc_classes=[]
for i in ids:
    lc_classes.append(ob_state[i])
lc_classes

drop_classes=[]
for clas, no in Counter(lc_classes).items():
    if no<7:
        drop_classes.append(clas)

lcs_abu = []
classes_abu = []
ids_abu = []
for n, lc in enumerate(lc_classes):
    if lc not in drop_classes:
        classes_abu.append(lc)
        lcs_abu.append(lcs[n])
        ids_abu.append(ids[n])

In [125]:
x_train, x_test, y_train, y_test, id_train, id_test = train_test_split(lcs_abu, classes_abu, ids_abu, test_size=0.5, random_state=0, stratify=classes_abu)

101